In [68]:
%matplotlib inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import numpy.linalg as la
np.set_printoptions(precision=3)
def pp_float_list(ps):#pretty print functionality
    return ["%2.3f" % p for p in ps]

In [69]:
#shell scripts for downloading the data and placing it in a corresponding directory
!mkdir CAR 
!curl -o CAR/data "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
!curl -o CAR/description "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.names"
#download the description and display it here.
!cat CAR/description

Ein Unterverzeichnis oder eine Datei mit dem Namen "CAR" existiert bereits.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 51867  100 51867    0     0  79474      0 --:--:-- --:--:-- --:--:-- 79672
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3097  100  3097    0     0   8795      0 --:--:-- --:--:-- --:--:--  8823
Der Befehl "cat" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [70]:
# csv-file has no header, so we define it manually
col_names = ['price_buy', 'price_main', 'n_doors', 'n_persons', 'lug_boot', 'safety', 'recommendation']
df = pd.read_csv("./CAR/data", header=None, names=col_names)

# All attributes are categorical - a mix of strings and integers.
# We simply map the categorical values of each attribute to a set of distinct integers
ai2an_map = col_names
ai2aiv2aivn_map = []
enc_cols = []
for col in df.columns:
    df[col] = df[col].astype('category')
    a = np.array(df[col].cat.codes.values).reshape((-1,1))
    enc_cols.append(a)
    ai2aiv2aivn_map.append(list(df[col].cat.categories.values))
    
# Get the data as numpy 2d-matrix (n_samples, n_features)
feature_names = ai2an_map
feature_value_names = ai2aiv2aivn_map[:6]
class_label_names = ai2aiv2aivn_map[6]

dataset = np.hstack(enc_cols)
X, y = dataset[:,:6], dataset[:,6]

In [71]:
def _entropy(p):
    """
    p: class frequencies as numpy array with np.sum(p)=1
    returns: impurity according to entropy criterion
    """
    idx = np.where(p == 0.) #consider 0*log(0) as 0
    p[idx] = 1.
  
    r = p * np.log2(p)
    
    return -np.sum(r)

def _gini(p):
    """
    p: class frequencies as numpy array with np.sum(p)=1
    returns: impurity according to gini criterion
    """
    return 1. - np.sum(p**2)

def _misclass(p):
    """
    p: class frequencies as numpy array with np.sum(p)=1
    returns: impurity according to misclassification rate
    """
    return 1-np.max(p)

In [72]:
def impurity_reduction(X, a_i, y, impurity, verbose=0):
    """
    Summary: This function evaluates the impurity reduction achieved by the 
    attribute `a_i` when the attribute were used for partitioning the dataset `X`.
    
    X: data matrix n rows, d columns
    a_i: column index of the attribute to evaluate the impurity reduction for
    y: class label vector with n rows and 1 column
    impurity: impurity function of the form impurity(p_1....p_k) with k=|X[a].unique|
    
    returns: impurity reduction
    """
    
    N, d = float(X.shape[0]), float(X.shape[1])

    y_v = np.unique(y)
    
    # Compute relative frequency of each class in X
    p = (1. / N) * np.array([np.sum(y==c) for c in y_v])
    # ..and corresponding impurity l(D)
    H_p = impurity(p)
    
    if verbose: print ("\t Impurity %0.3f: %s" % (H_p, pp_float_list(p)))
    
    # a_v is an array of unique values in column a_i
    a_v = np.unique(X[:, a_i])
    
    """
    Create and evaluate splitting of X induced by attribute a_i
    We assume nominal features and perform m-ary splitting.
    
    We partition the dataset X into as many subsets as there are different values
    in attribute a_i. Each subset contains the exactly those data points that have 
    the value `a_vv` in attribute `a_i`. The following steps are required:
    - Once we have determined which data points belong to a subset (`mask_a`), 
      we can look up their corresponding class labels in the vector of 
      class labels `y`.
    - Then we count how many times each class label appears in the subset and
      calculate the relative frequencies `pa`.
    - Now we can calculate the impurity of the subset `H_pa_vv` based on the 
      relative frequencies `pa` of the classes that are present in the subset.
    - We add the impurity of the subset `H_pa_vv` to a list `H_pa`.
    - ...and repeat the process until we have evaluated all subsets

    """
    
    H_pa = []
    for a_vv in a_v:
        mask_a = X[:, a_i] == a_vv
        N_a = float(mask_a.sum())
                     
        # Compute relative frequency of each class in X[mask_a]
        pa = (1. / N_a) * np.array([np.sum(y[mask_a] == c) for c in y_v])
        H_pa_vv = (N_a / N) * impurity(pa)
        H_pa.append(H_pa_vv)
        if verbose: print ("\t\t Impurity %0.3f for attribute %d with value %s: " % (H_pa[-1], a_i, a_vv), pp_float_list(pa))
    
    """
    The impurity reduction (aka "information gain" if we use entropy as a measure),
    is given by the difference of the impurity of the whole dataset `H_p` and the 
    weighted sum of the impurities of the subsets `np.sum(H_pa)`.
    """
    IR = H_p - np.sum(H_pa)
    if verbose:  print ("\t Estimated reduction %0.3f" % IR)
    return IR

In [73]:
def get_split_attribute(X, y, attributes, impurity, verbose=0):
    """
    X: data matrix n rows, d columns
    y: vector with n rows, 1 column containing the class labels
    attributes: A dictionary. The key corresponds to an attribute's column index in X and the value is a list of values the attribute can take (its domain).
    impurity: impurity function of the form impurity(p_1....p_k) with k=|y.unique|
    returns: (1) column idx of attribute with maximum impurity reduction and (2) impurity reduction
    """
    if type(X) == "numpy.ndarray":
        N, d = X.shape
    else:
        N, d = len(X), len(X[0])
    IR = [0.] * d
    
    # TODO
    for i,key in enumerate(attributes.keys()):
        IR[i] = impurity_reduction(X,key,y,impurity)
    
    return np.argmax(IR), np.max(IR)

In [74]:
def most_common_class(y):
    """
    :param y: the vector of class labels, i.e. the target
    returns: (1) the most frequent class label in 'y' and (2) a boolean flag indicating whether y is pure
    """
    
    # TODO
    ctemp = {}
    for c in y:
        if ctemp.__contains__(c):
            ctemp[c] += 1
        else:
            ctemp[c] = 1
    
    maxi = max(ctemp, key = ctemp.get)

    return maxi, ctemp[maxi] == len(y) # len(ctemp) == 1 

In [95]:
class DecisionNode(object):
    NODEID = 0
    
    def __init__(self, attr=-1, impurityReduction=-1, children=None, label=None, isPure=False, X=None, y=None, isPerfectSplit=False, maxDepth=5):
        self.attr = attr # attribute with maximum impurity reduction before split
        self.impurityReduction = impurityReduction # the amount of impurity reduction down from the parent
        self.children = children # list of new nodes, split according to impurity
        self.label = label # label with the most occurences in this node
        self.isPure = isPure # tells whether this node contains just 1 class
        self.id = DecisionNode.NODEID # node id
        DecisionNode.NODEID += 1 # id counter
        self.X = np.array(X)
        self.y = np.array(y)
        self.isPerfectSplit = isPerfectSplit
        self.maxDepth = maxDepth
        
        
    def make_children(self):
        self.combinedXy = np.concatenate(((self.X, self.y.reshape(-1,1))), axis=1) # X with an extra column for y
        self.adjustedXy = np.array([[row for row in self.combinedXy if row[self.attr] == each] for each in np.unique(self.X[:,self.attr])]) # 3d np array where the data is split according to max impurity reduction
        self.adjustedX = [np.concatenate((splitSet[:,0:self.attr], splitSet[:,self.attr+1:-1]), axis=1) for splitSet in self.adjustedXy] # list of subsets of X without the splitting column and y
        self.adjustedy = [splitSet[:,-1] for splitSet in self.adjustedXy] # list of classes (y)
        self.adjustedattributes = {k: None for k in range(len(self.adjustedX[0][0]))} # list of possible attributes to split
        
        tempChildren = []
        for i in range(len(self.adjustedX)):
            attribute, impurityReduction = get_split_attribute(self.adjustedX[i],self.adjustedy[i],self.adjustedattributes,_entropy)
            label, isPure = most_common_class(self.adjustedy[i])
            tempChildren.append(DecisionNode(attribute, impurityReduction, None, label, isPure, self.adjustedX[i], self.adjustedy[i]))
        self.children = tempChildren
        #[print(childe) for childe in self.children]
        
    def make_grandchildren(self):
        DecisionNode.helper(self.children)

        
    @staticmethod
    def helper(agenda):
        tempAgenda = []
        for child in agenda:
            if not child.isPure and len(Root.X[0]) - len(child.X[0]) < child.maxDepth:
                child.make_children()
            if child.children != None:
                [tempAgenda.append(childe) for childe in child.children] 
        if len(tempAgenda) > 0:
            DecisionNode.helper(tempAgenda)        
        else:
            return 
           
    def __str__(self): # toString()
        result = str((
                    feature_names[self.attr], 
                    self.impurityReduction.__format__(".3f"), 
                    self.children, feature_names[self.label], 
                    "Pure" if self.isPure else "Impure", 
                    f"{np.sum([1 for label in self.y if label == self.label])}/{len(self.X)}", 
                    self.id
                    ))
        
        if self.id == 0: result += " - Root"
        elif self.id < 4: result += " - Children"
        else: result += " - Grandchildren"
        
        if self.id > 0: return result
        else: return "# splitting Attribute, Impurity Reduction, Children, majority Label, isPure, Size, ID\n" + result

# initial Root setup        
attribute, impurityReduction = get_split_attribute(X, y, {k: None for k in range(X.shape[1])}, _entropy)
label, isPure = most_common_class(y)
Root = DecisionNode(attribute, impurityReduction, None, label, isPure, X, y)
print(Root)
# Root setup end

Root.make_children()
Root.make_grandchildren()
print(DecisionNode.NODEID)

# splitting Attribute, Impurity Reduction, Children, majority Label, isPure, Size, ID
('safety', '0.262', None, 'n_doors', 'Impure', '1210/1728', 0) - Root
222


"""
Ex06
1
Model Stacking:
use different learning algorithms on the data and afterwards a combiner with the results of the previous learners as inputs
Boosting:
train different learners sequentially and focus on the wrong results by using different weights in the latter learners
Bagging:
sample different sets from the data and train multiple learners on it. afterwards make a majority vote from the results

2
while bagging is an idea of a learning algorithm, random forests are an explicit instance. different trees are being trained with random subsets of attributes

3a
client is atomic and unique and can perfectly determine the risk but is just an id, from which nothing can be exrapolated or learned
3b
TODO?

4
the set needs to have mixed classes on x1 axis at the same x2 value
""" 